## Kaggle.Kernels have some problems with widgets in the view mode, so you can't see the interactive widget untill you'll fork the notebook.
## Let's hope the kaggle team will fix it soon (or I'll find a solution)))

Hello, I'm doing some EDA, so I explored very curious ways other participants check raw data, with grids etc.Nevertheless, I made utility for this task, thanks very much to https://www.kaggle.com/aleksandradeis/globalwheatdetection-eda for baseline)
Its advantage is in space economy, so you can see a lot of images without mud in your code

If notebook it'll earn some upvotes, I'll post more utils

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ipywidgets import interact, IntSlider, fixed
from pathlib import Path
from PIL import Image

In [ ]:
base_path = Path('../input/global-wheat-detection/')
TRAIN_CSV_PATH = base_path / 'train.csv'
TRAIN_DIR = base_path / 'train/'

## utility:

In [ ]:
def get_all_bboxes(df, img_id):
    image_bboxes = df[df.image_id == img_id]      
    return image_bboxes[["x", "y", "w", "h"]]

def plot_image(df, img_id, train_dir=TRAIN_DIR): # using constant, be careful
    plt.figure(figsize=(9, 9))
    ax = plt.gca()
    
    img = Image.open(str(train_dir / (img_id + '.jpg')))
    ax.imshow(img)
    
    #plot bboxes on image
    bboxes = get_all_bboxes(df, img_id)
    for bbox in bboxes.iterrows():
        bbox = bbox[1] # 0 element is index
        coords = ((bbox.x, bbox.y), bbox.w, bbox.h)
        rect = patches.Rectangle(*coords, linewidth=1, edgecolor='r',facecolor='none')
        ax.add_patch(rect)

    plt.axis('off')

In [ ]:
def interactive_plot(df):
    uniq = df["image_id"].unique()
    @interact(df=fixed(df), id_num=IntSlider(max=len(uniq)-1))
    def choose_show(df, id_num):
        img_id = uniq[id_num]
        plot_image(df, img_id)

primary preprocessing function for demonstrating of an example (you can open code)

In [ ]:
def bbox2separate_cols(df):
    """
    If df contains bbox column of 'O' dtype, function creates columns "x", "y", "w", "h". 
    Otherwise throws ValueError.
    """
    try:
        bbox_items = df["bbox"].str.split(',', expand=True)
    except KeyError:
        raise ValueError("df has no bbox col")
        
    # convert splitted strings to numbers
    bbox_items[0] = bbox_items[0].transform(lambda x: x[1:]) #delete '[' and ']' symbols
    bbox_items[3] = bbox_items[3].transform(lambda x: x[:-1])
    bbox_items = bbox_items.astype(float)
    
    train[['x', 'y', 'w', 'h']] = bbox_items
    train.drop(columns=['bbox'], inplace=True)
    
    return train

## example of use

In [ ]:
train = pd.read_csv(TRAIN_CSV_PATH)
train = bbox2separate_cols(train)

### So, if you view this notebook in commit (not edit) mode, you probably can't see the widget that allows you to scroll through image ids and check all images. You still can copy notebook and use utility in your project)

In [ ]:
interactive_plot(train)